In [1]:
# load libraries
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
from PIL import Image

C:\Users\danie\anaconda3\envs\ds_master\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Sample usage

source = https://huggingface.co/arnabdhar/YOLOv8-Face-Detection

In [2]:
# download model
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")

# load model
model = YOLO(model_path)

# inference
#image_path = "sample/2013-10-08_12-40-36_UTC.jpg" # face
image_path = "sample/2022-11-30_16-58-45_UTC_6.jpg" # no face
image = Image.open(image_path)

# Perform inference
output = model(image, save_conf=True)
results = Detections.from_ultralytics(output[0])


0: 640x512 (no detections), 212.9ms
Speed: 10.1ms preprocess, 212.9ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 512)


# Label all data

In [3]:
import os
import glob

# Replace this with the directory you want to search in
directory = '../1_download_data/data/raw/' #change when data is new data



def get_jpg_files(folder_path):
    jpg_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.jpg'):
                jpg_files.append(os.path.join(root, file))
    return jpg_files

# Example usage
folder_path = '../1_download_data/data/raw/'
jpg_files = get_jpg_files(folder_path)
jpg_files[0:10]

['../1_download_data/data/raw/-PHbiexlr_\\2015-11-18_18-26-59_UTC.jpg',
 '../1_download_data/data/raw/3YO1o0Rlkt\\2015-06-01_07-45-08_UTC.jpg',
 '../1_download_data/data/raw/4yqgDyxloK\\2015-07-06_10-38-30_UTC.jpg',
 '../1_download_data/data/raw/B-y2Fo8qcN_\\2020-04-10_08-57-20_UTC.jpg',
 '../1_download_data/data/raw/B3__d73i46x\\2019-10-24_11-49-40_UTC.jpg',
 '../1_download_data/data/raw/B8zBgZeD8bQ\\2020-02-20_17-34-19_UTC.jpg',
 '../1_download_data/data/raw/B9UltWvIDjA\\2020-03-04_18-25-36_UTC.jpg',
 '../1_download_data/data/raw/B9XY-IjjDWt\\2020-03-05_20-32-01_UTC_1.jpg',
 '../1_download_data/data/raw/B9XY-IjjDWt\\2020-03-05_20-32-01_UTC_2.jpg',
 '../1_download_data/data/raw/BBIHpekRlhD\\2016-01-29_14-48-24_UTC.jpg']

In [6]:
from tqdm import tqdm
dict_face_detection = {}
for image_path in tqdm(jpg_files[0:10]):
    short_name = image_path.split('\\')[1]
    try:
        image = Image.open(image_path)
        # Perform inference
        output = model(image)
        results = Detections.from_ultralytics(output[0])
        tensor = output[0].boxes.xyxy  # Replace with your tensor
        # Accessing the first element
        if tensor.numel() > 0:  # Check if the tensor is not empty
            dict_face_detection[short_name] = 1
        else:
            dict_face_detection[short_name] = 0
    except:
        pass
 

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]


0: 640x640 2 FACEs, 149.6ms
Speed: 5.5ms preprocess, 149.6ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)


 10%|████████▎                                                                          | 1/10 [00:00<00:02,  4.28it/s]


0: 640x640 1 FACE, 122.7ms
Speed: 0.0ms preprocess, 122.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


 20%|████████████████▌                                                                  | 2/10 [00:00<00:01,  5.45it/s]

...

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [00:01<00:00,  5.23it/s]


0: 640x512 (no detections), 106.6ms
Speed: 4.0ms preprocess, 106.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 512)


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:01<00:00,  5.51it/s]


0: 640x640 1 FACE, 130.6ms
Speed: 7.5ms preprocess, 130.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.34it/s]


In [7]:
dict_face_detection

{'2015-11-18_18-26-59_UTC.jpg': 1,
 '2015-06-01_07-45-08_UTC.jpg': 1,
 '2015-07-06_10-38-30_UTC.jpg': 1,
 '2020-04-10_08-57-20_UTC.jpg': 0,
 '2019-10-24_11-49-40_UTC.jpg': 1,
 '2020-02-20_17-34-19_UTC.jpg': 1,
 '2020-03-04_18-25-36_UTC.jpg': 1,
 '2020-03-05_20-32-01_UTC_1.jpg': 0,
 '2020-03-05_20-32-01_UTC_2.jpg': 0,
 '2016-01-29_14-48-24_UTC.jpg': 1}

In [10]:
import pandas as pd
df_faces = pd.DataFrame(dict_face_detection.items(), columns=['Long', 'Face'])
df_faces

,Long,Face
0,2015-11-18_18-26-59_UTC.jpg,1
1,2015-06-01_07-45-08_UTC.jpg,1
2,2015-07-06_10-38-30_UTC.jpg,1
3,2020-04-10_08-57-20_UTC.jpg,0
4,2019-10-24_11-49-40_UTC.jpg,1
5,2020-02-20_17-34-19_UTC.jpg,1
6,2020-03-04_18-25-36_UTC.jpg,1
7,2020-03-05_20-32-01_UTC_1.jpg,0
8,2020-03-05_20-32-01_UTC_2.jpg,0
9,2016-01-29_14-48-24_UTC.jpg,1


In [ ]:
df_faces.to_excel('../0_data/dataframe_face_detection.xlsx', index=False)